In [1]:
import pyspark.sql.functions as F
import pandas as pd
import xlrd
import requests
import json
import logging 
import datetime

from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import *
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

In [2]:
file_name='/home/cm2506/pysparknb/melissa_geocoder.pdf'
url = 'https://wiki.melissadata.com/index.php?title=GeoCoder_Object'


datetime_object = datetime.datetime.now()
date=str(datetime_object).split(' ')[0]
time=str(datetime_object).split(' ')[1]

log_path='/home/cm2506/Documents/logs/'
log_file_name=log_path+'newfile_'+date+'.log'


spark = SparkSession \
    .builder \
    .appName("schema Comaprsion Df") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.executor.instances","10") \
    .enableHiveSupport()\
    .getOrCreate()

logging.basicConfig(filename=log_file_name, 
                    format='%(asctime)s %(message)s', 
                    filemode='w') 

#Creating an object 
logger=logging.getLogger() 
logger.setLevel(logging.INFO)

In [3]:
def download_content(url,file_name): 
    """Method to download the contents from webpage"""
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()    
    page_soup = soup(webpage, "html.parser")
    containers = page_soup.findAll("div","padded")
    for container in containers:
        hyperlinks=container.findAll('a')
        for hyperlink in hyperlinks:
            download_url=hyperlink.get('href')
            if download_url.endswith('DQT_API_GeoCoder_RG.pdf'):
                download_url='https://wiki.melissadata.com'+download_url
                #print (download_url) 
                logger.info(str(download_url))
                r = requests.get(download_url, stream = True)                
                with open(file_name, 'wb') as f:
                    for chunk in r.iter_content(chunk_size = 1024*1024):
                        if chunk:
                            f.write(chunk)
                            download_path='File is downlaoded in : {0} '.format(file_name)
                            logger.info(download_path)

In [4]:
def extract_content(url):
    ip_file_name='/home/cm2506/Documents/data/input.xlsx'
    #download_content(url,file_name)
    logger.info(file_name)
    column_list=[]
    type_list=[]
    
    abosulte_file_path = (ip_file_name) 
    logger.info('abosulte_file_path :')
    logger.info(abosulte_file_path)
    
    wb = xlrd.open_workbook(abosulte_file_path) 
    sheet = wb.sheet_by_index(0) 
    sheet.cell_value(0, 0) 

    rows='Number of rows extarcted : {0} '.format(sheet.nrows)
    logger.info(rows)
    cols='Number of cols extarcted : {0} '.format(sheet.ncols)
    logger.info(cols)
    
    for i in range(sheet.ncols): 
        col_name=sheet.cell_value(2, i)
        if col_name =='column_name':
            column_name=col_name
            for j in range(sheet.nrows):
                #print(sheet.cell_value(j, 4)) 
                if j >=3:                
                    col_value=str(sheet.cell_value(j, 4)).lower()
                    column_list.append(col_value)            
        if col_name == 'column_type':
            column_type=col_name
            for j in range(sheet.nrows):
                #print(sheet.cell_value(j, 5))           
                if j >=3:
                    type_list.append(sheet.cell_value(j, 5))

    d= {'col_name': column_list,'data_type':type_list}
    logger.info(d)
    df = pd.DataFrame(d, columns = ['col_name', 'data_type'])
    logger.info (type(df))
    
    pdf = spark.createDataFrame(df)
    logger.info(type(pdf))
    
    pdf.registerTempTable('cmp_source_tbl')
    df2=spark.sql("select col_name,data_type from cmp_source_tbl")
    df2.show()
    
    
    #use_db=spark.sql('use cm2506')
    db_name='cm2506'
    table_name='target_tbl'    
    db_name='cm2506'
    db_name='cm2506'
    logger.info('db_name : {} tablename: {}'.format(db_name,table_name))
    query='desc {0}.{1}'.format(db_name,table_name)
    show_info=spark.sql("desc cm2506.target_tbl")
    show_info.registerTempTable('cmp_target_tbl')
    
    df1=spark.sql("select col_name,data_type from cmp_target_tbl")
    logger.info(type(df1))
    
    
    cmp=spark.sql('select a.*,case when a.col_name=b.col_name then "Matched" else "Unmatched" end as matching_ind from cmp_source_tbl a join cmp_target_tbl b on a.col_name = b.col_name')
    cmp.show()
    
    cmp.toPandas().to_csv('output.csv')
    logger.info('Success!!')
    

In [5]:
if __name__=="__main__": 
    download_content(url,file_name)
    extract_content(url)

+-----------------+---------+
|         col_name|data_type|
+-----------------+---------+
|         geocode |   string|
|        geopoint |   string|
|        cbsacode |   string|
| cbsadivisioncode|   string|
|cbsadivisionlevel|   string|
|cbsadivisiontitle|   string|
|       cbsalevel |   string|
|       cbsatitle |   string|
|      censusblock|   string|
|      censustract|   string|
|       countyfips|   string|
|       countyname|   string|
|       errorcode |   string|
|         latitude|   string|
|        longitude|   string|
|       placecode |   string|
|       placename |   string|
|      statuscode |   string|
|         timezone|   string|
|     timezonecode|   string|
+-----------------+---------+
only showing top 20 rows

+-----------------+---------+------------+
|         col_name|data_type|matching_ind|
+-----------------+---------+------------+
| cbsadivisioncode|   string|     Matched|
|cbsadivisionlevel|   string|     Matched|
|cbsadivisiontitle|   string|     Match